# Embedded kernels 

This notebook and the C# project in this folder demonstrates how you can use .NET Interactive to embed a kernel within an app, connect to it from another kernel, and use the notebook to change the app's runtime state.



## Connect to the WPF app

First, let's start the WPF app and connect to it.

In [1]:
Start-Process -NoNewWindow dotnet run

Once the cell above has finished running, you should see the app's window open. Next, we'll connect to it using a named pipe. The code that sets this up within the WPF app can be seen in [`App.xaml.cs`](https://github.com/dotnet/interactive/blob/main/samples/connect-wpf/App.xaml.cs).

In [3]:
#i "nuget:I:\oss\dotnetinteractive\artifacts\packages\Debug\Shipping"
#r "nuget:Microsoft.DotNet.Interactive.Documents,*-*"
#r "nuget:Microsoft.DotNet.Interactive.NamedPipeConnector,*-*"
#!connect named-pipe --kernel-name wpf --pipe-name InteractiveWpf

Kernel added: #!wpf

The topology of connected kernels now looks like this:

In [3]:
flowchart LR
    subgraph WPF app
    embedded["Embedded C# kernel"]
    end
    subgraph notebook
    CompositeKernel-->n1["Local C# kernel"]
    CompositeKernel-->n2
    n2["#!wpf kernel added using #!connect"]--named pipe-->embedded
    end

## Change the styling of the app

The notebook outputs here are displayed using custom formatters defined within the WPF app itself. Take a look at the file [`WpfFormatterMixins.cs`](https://github.com/dotnet/interactive/blob/main/samples/connect-wpf/WpfFormatterMixins.cs).

You'll also notice that you can get completions for the `App` object which is exposed to the notebook's kernel by the embedded kernel. 

In [5]:
#!dispatcher
using System.Windows.Media;

App.MainWindow.Background = new SolidColorBrush(Colors.Fuchsia);
App.MainWindow.Background

#FF00FFFF

In [8]:
#!dispatcher
using System.Windows.Media;
using System.Windows.Controls;
using System.Windows;

var content = (Grid)App.MainWindow.Content;
content.Background = new SolidColorBrush(Colors.RoyalBlue);
content.UpdateLayout();
content

<img src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAxEAAAGcCAYAAACm1i1JAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAADsMAAA7DAcdvqGQAABTbSURBVHhe7d17kF5lfcDxcwgkhiQkIffEJBJMiAkESLjEqFxUWppBAbXQUgVrLZapdeponbEd7T9tdZxxRp1W6x0sYxVBTaUMioKBCJGiMQRCCCQQAgnkHnKDXDjN8+bZl3c3e/uFNbsvfj4z7/Ccs8vZPZv3j/N9z+UpL7xuTVUAAAB00zH5vwAAAN0iIgAAgBARAQAAhIgIAAAgREQAAAAhIgIAAAgREQAAQIiIAAAAQkQEAAAQIiIAAIAQEQEAAISICAAAIKS88Lo1VR7ze/L0suurxxb9c5kXi4N/8zx69ftD3ncAgFerPncmYsOq26qFX59e/fI/p1TLf/H3PRo4OzY+VC26fnZ111cmF0sWXFm9dOBFAQUAAEGtIiJ9apwOsNPr3v+a1ysH2Kvu+7fipf17yqo6UD638kfl9vUP9NjvsWrxZ4p9ezbXPhXftm5xuW75f9fWN4v1K26q//uk15rf/scr+tukiEqx1rK9no42AABenfr8PRH79+3Mo553YN+uPGoO46ZfUR73muH1A/1NT9yRR0dm7dJvFCnW8mIxYcZ78wgAADrW5yLi5Df+Y3HMsQOrsuxXjZl2eTVi0gX1g9y20ifpzz76w+qBm99Rrbz7U11+in7y3E8Wxw0cUfu+YePnVhNP/2BtfTM5cdIFeVQUz29YUu7dvemIzx40RsjAoVOqoePO6vBvDQAALfpcRIw+eX55/l+vKC/4m9XljLd9odOD2qW3XlM8cudHyx0bHyyran9e27Eho04t3/z+36abyYszL/1+eUy/AU130Dxm6mV5dMi65d/No5gUHylC8mIx8qSL8ggAADrX5y9norV0ZmbA4An1sw/PPf4/eRTz1NKv5VFRpLM+k06/Ni8BAEDnREQTGjXl4jwqit1bHyt3b1sdvqSp8VKmIaNmFf2PH+lSJgAAukVENKEJM1vfAP3MwzfmUffs3Lyi2rN9dT0axs+8Ko8AAKBrIqIJHT9sSjlk1Gn1sw+b19yVR93zzEM35NHBN8CxA6sxUy/NSwAA0LVWM1Y3zi6crruf9757273EJc3d8Nsfv7v+tdmX3VJ/ss/6R2+u1v7uq8XubU8W1Ut7a+uOHTC0GjHpwmLaef9aHNt/cLvbbPHowk9W65Z/t/Y9bX+Htj+3K+NnXFWdcv5n6t8fnT15/96d1bqHbyw2rPrfYs/za4r9L26v/79pn04YfUYxefbfFsPGn9vp7/T7mLU5zRGx+tefq2+z8d+gK4uun1Pt27Op9r2jX/+OauZF/37Y/3e09z3NU9Giu/uyZMGfVdvW3Vf7vrb/1h1JT/Ra98j3irRve55fW5uTJK1PMTXwhInF+JnvLV576jVdbgcA4A9Zj52JeGHnuur+my6uVtz5sXLXlpVlS0Ak6QD0ucd+XC769pm1Ganz6j4rHWguu/1D1aJvzUoT1NWe/tR4EJ2k5S1rF5ZLFlxRPnjbXx31fUqPp003ROfF4tmVt+RR5zY/9ct6QCRjT3lPHh3SDPt+pNJ771c3nF08ds+na+/RloBI0jitS1+79ztzq93bn2ya/QIAONp6JCJeemlvseTHVxS7Nj9SOyhLn1QPGTWrShOjNR7oprBYfseHa9fk51UhZb/jirTNlld5TP/6dtInyY1fS69+xw3OX43ZsWFZsemJ22uzZudVxcFtVQMGja2Gjju7tu28umbzmp+X6QxKXjwq0uNph088Ly8VxcYnfpZHnXv20Zvz6NCZnrbzcDTDvh+JdEbk4Z9d1yqIWt6nA4e+rkrvn7y6eHHX+vKBmy+phXFeBQBAgx6JiJX3fLp4YcfacvDImdW5Vy2s3vKBB8uz3vOT8s1/+bvyzR94sBgx+e31g7F0cLpq8WfzUswJo08/tM38Gjp2Tv5KUYyddnnR+LX0ev28f2p1gByVDjInz/m76k3vX1Kd98GHy3lX/7qcfdnNtW2ndYNGvKG+X+sf+X5t7oW8eFSMPnl+HhVFOruQzjLkxQ5tevLnedT6KU9t9fV9j9j05B2tLqlq+z6de9XC2twkM97+papf/yG1/Tiwd0e59Nara98PAEBrPRIR6TGjw8a/sTr7T28rjx/6ulYH7ukeiFnzv1mmT7HzqmLr2rv79EFnkmbLftM1/1dMOefjZf+BJx4WI2ndGZfcWDsDkpZTHDXOvXA0jJt+Rdn4CXrjWYb2rF9xU9V4CU/bpzy1aIZ97650edbyX3w0Lx26B6S992kyZuql5WkXf61+mVh6X6e/We2LAADU9UhEpE9vT7/k5Sf+tGfy7A/n0aGDzi1P35OX+p50U2+aLburGa3T3ArDJ7wpLxXFzk0P59HR03g2YevTi/KofRtW3ZZH6azOmVV6ylNerGumfe+O1fd/vnZWIY3T5VtveOvna+s7MnzCvHL01HfmpaLYuPqneQQAQIseiYgxr39H7Rr9vNiudO1946fmOzc/kkfNbcCg0XmU7vk4kEdHz5ipl+VRUex7YWv57MoftfvJefpEPp0BajHypIvy6Mj19r53x+Y1d+ZRUUw649ou36fJiRPPz6MURw/lEQAALXokIkZMfmseda7/wJF5VLvmPI+aw66tj1cbV99erb7/89XSW99XLf7u+dU935pVfxxtb0lxlj5hz4vFc48tyKPW1i79Ru0MUBqnmEtPd+quvrrvXUmXzKVLkvJiMfrkS/KocwOHTMyjdJP1s316HwEAekOPRMRxA4bnUedeM+S1edQcnn7ohtoBc5rD4P7vva186KcfKtf85kvllrV3l3u2P3nYo097S+MlTdvWLa6ddciLdZueuCOPimLY+LldfiLfLPvemYO/Zx4d8qsb5pRpf7p6tZ2L5PkNS90XAQDQoEci4tUmzRHwqxvOqtKcAemAOa9uJT3qdNCJ06rXDJnY6weYjTdIpxun256NSJ/IP79hSX0/Jsz8izw6XLPte2d29NClSNWBfXkEAEAiItpIB9xpjoC9uzfWD6DTAfPkOR+p5rxrQe2Rp2nW5fSo03OuvKM8ceJb8nf1nnSD9JBRp9UP6J999Id5dMiaJV/Oo4MBMHBkNfJ1F7UbB8247xEpfo7kleYnAQDgZSKijcfv+5f603zSU6dmv+tHVTpgnnLOx8oTxpzR7iNP+4JRU/4kj4pi+/r7a0GQF4vNa+7Ko4Pfd9If5dHhmnXfO3IwgPLokBQ/R/JK85PkTQAAcJCIaGPLU7/Mo6KYfOZ1xdAxszs9gOwrTyUaN/3K+vwG6Qbq9Su+X1u/ff0D1Z7tq+v7MOHUa/LocH1l3/e9uDWPOndg3848at+AQWPz6JDd21b36cuvAACahYhoIz0mNQ+LYePOzaOO7dy8PI96V23ehonn5aWXb6R+ZvmNtf8mxw+fWg0eMb3DMOjNfW/1+N9NXT/+N908vmvr43mpfekyr3Q5Ul4sNjx+ax4BAPBKvGoiojfOCGx95t5qx8ZlnX5afzSNPnl+HqUnCi0p0yVNW9bek9cUxdhpl+fRK9fT+9745K7GuR068tSSr9ZuIs+LHTphzJw8qj1xqt0nVwEAENPUEVEe0y+Peu5T8ZZLgpINq1+e4bmt/Xt3Vivu+kRe6hvGTb+i1YR+Tz7wxWLfnk21A+20X13NDdGb+z5s3Nl5dCiAUqTkxcOkJ0g9tfRrealz6bKsFulvsfTWji/narRj40PVI3d+THAAALSjqSNi8MiZeZQO+paV61f84BUf9A0ZNSuPiuKZZdcXa5Z85bBtblv36+q+G+cVL+xYW6anHeXVfULjnBEbVv0kj4oiXerU1dwQvbnvbe/VWHb7tWnivMO2n9alJ0ilG8C78/OHjjurHDH57fXv27buvvLe78yt0n7

## Change view models at runtime



Create and apply a new view model to the main window.

In [9]:
using System.ComponentModel;
using System.Collections.ObjectModel;

public class TestViewModel : INotifyPropertyChanged
{
    public event PropertyChangedEventHandler PropertyChanged;

    private string _text = "Initial Value from notebook view model";
    
    public string Text
    {
        get => _text;
        set
        {
            if (_text != value)
            {
                _text = value;
                PropertyChanged.Invoke(this, new PropertyChangedEventArgs(nameof(Text)));
            }
        }
    }
}

var vm = new TestViewModel();

#!wpf
#!dispatcher
App.MainWindow.DataContext = vm;

Update the value on the data bound property.

In [10]:
vm.Text = "Value changed!"

Value changed!

 ## Dispatcher stuff

 Demonstate enabling and disabling running code on the dispatcher. 

In [13]:
#!dispatcher --enabled 
//This should work
App.MainWindow.Title = "Title change executed on dispatcher thread";


In [14]:

#!dispatcher --enabled false
//This is expected to fail
App.MainWindow.Title = "Not so much";

Error: System.InvalidOperationException: The calling thread cannot access this object because a different thread owns it.
   at System.Windows.Threading.Dispatcher.<VerifyAccess>g__ThrowVerifyAccess|7_0()
   at System.Windows.Application.get_MainWindow()
   at Submission#15.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)